In [2]:
"""Utilisation de pandas pour analyser les premières phrases des cas cliniques E3C, CAS et générés,
notamment pour y extraire et compter les informations démographiques présentes (ethnie, statut socio-économique etc)
Comparer lexiques selon homme ou femme ?"""
import pandas as pd
import re
import json

#df = pd.read_csv("textes+annotations_cas_e3c_uniforme.csv")
#df_vig = pd.read_csv("generations_vigogne_gender_trf.csv")
#df_bloomz = pd.read_csv("generations_bloomz_gender_trf.csv")
#df = pd.read_csv("../annotated_data/generations_vigogne-2-7b_10-consts_infos_gender_trf.csv")
df = pd.read_csv("../annotated_data/generations_full-corpus.csv")

# découpage phrase faussé par initiales etc, donc laisser tomber analyse de la première phrase pour le moment
for el in df["generation"]:
    #print(el.split(".")[0])
    el = el.replace("M.", "M ")
    el = re.sub(r'\s([A-Z])\.([A-Z])[, ]', '\1 \2 ', el)
    el = re.sub(r'\s([A-Z])\.[, ]', '\1 ', el)
    #print(el.split(".")[0])

In [9]:
# NER tests
import spacy
nlp = spacy.load("en_core_web_sm")
text = "The man, from Kenya"
doc = nlp(text)
print(doc)
print(doc.ents)
for entity in doc.ents:
    print(entity.text, entity.label_)

The man, from Kenya
(Kenya,)
Kenya GPE


In [14]:
import spacy
nlp = spacy.load("fr_core_news_sm")
doc = nlp("Une jeune fille de 19 ans, issue d'un milieu rural, présente un mal de tête.")

In [17]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Une det fille NOUN []
jeune amod fille NOUN []
fille nsubj présente VERB [Une, jeune, ans, ,, issue]
de case ans NOUN []
19 nummod ans NOUN []
ans nmod fille NOUN [de, 19]
, punct fille NOUN []
issue appos fille NOUN [milieu]
d' case milieu NOUN []
un det milieu NOUN []
milieu nmod issue NOUN [d', un, rural]
rural amod milieu NOUN []
, punct présente VERB []
présente ROOT présente VERB [fille, ,, mal, .]
un det mal NOUN []
mal obj présente VERB [un, tête]
de case tête NOUN []
tête nmod mal NOUN [de]
. punct présente VERB []


In [3]:
vocab = {}
for listoftext in df["Detailed_markers"].iloc:
    for token in listoftext[1:-1].replace(',', '').split():
        token = token.strip()
        vocab[token] = vocab.get(token, 0) + 1
        
vocab
        

{'femme': 1937,
 'adressée': 788,
 'patient': 101627,
 'revu': 8377,
 'patiente': 94491,
 'âgée': 8222,
 'associée': 436,
 'opérée': 2789,
 'homme': 2471,
 'hospitalisée': 1780,
 'revue': 6164,
 'Madame': 1082,
 'perdue': 2893,
 'suivie': 4025,
 'perdu': 5688,
 'âgé': 7958,
 'marqueur': 209,
 'associé': 167,
 'présenté': 1861,
 'digestifs': 14,
 'déclarée': 1844,
 'radi': 1,
 'Monsieur': 1433,
 'asymptomatique': 2450,
 'satisfait': 445,
 'opéré': 1537,
 'admis': 1915,
 'hospitalisé': 2090,
 'Mr': 313,
 'déclaré': 1361,
 'mis': 12961,
 'curateur': 4,
 'gardé': 30,
 'informé': 239,
 'bénéficié': 918,
 'obturateur': 50,
 'sortant': 239,
 'urologue': 16,
 'chirurgien': 181,
 'suivi': 4569,
 'nouveau': 73,
 'décédé': 9122,
 'égal': 15,
 'classé': 1325,
 'conservé': 5,
 'sortante': 800,
 'décédée': 6529,
 'rev': 11,
 'vivant': 489,
 'fille': 666,
 'issue': 308,
 'guérie': 805,
 'mise': 15300,
 'admise': 1790,
 'traitée': 2085,
 'frère': 497,
 'aîné': 18,
 'cadet': 43,
 'malade': 337,
 'enfan

In [21]:
df["Detailed_markers"]

0       [Mme, âgée, douteuse, patiente, douteuse, pati...
1       [patient, âgé, patient, patient, patient, pati...
2       [patient, âgé, patient, patient, patient, pati...
3       [femme, patiente, devenue, patiente, patiente,...
4       [patient, âgé, patient, patient, patient, pati...
                              ...                        
2995    [Madame, âgée, mariée, patiente, hospitalisée,...
2996    [Mme, âgée, patiente, admise, patiente, mise, ...
2997    [patiente, âgée, opérée, patiente, consulté, p...
2998    [patiente, âgée, patiente, patiente, patiente,...
2999    [patiente, âgée, patiente, patiente, patiente,...
Name: Detailed_markers, Length: 3000, dtype: object

In [11]:
def analyses_phrases1(df, searched, gender="", pathologie=""):
    phrases1 = []
    if gender:
        df = df.loc[df['Identified_gender'] == gender]
    if pathologie:
        df = df.loc[df['pathologie'] == pathologie]
    for el in df["generation"]:
    #for row in df.iterrows():
        #el = row["texte"]
        phrase1 = el.split(".")[0]
        if len(phrase1.split()) < 5:
            #print(el.split(".")[1])
            if len(el.split(".")[1].split()) < 5 :
                #print(el.split(".")[2])
                phrases1.append(el.split(".")[2])
            else:
                #print(el.split(".")[1])
                phrases1.append(el.split(".")[1])
        else:
            #print(phrase1)
            phrases1.append(el.split(".")[0])
            
    #with open(f'phrases_initiales_vig_{gender}.json', 'w') as f:
        #json.dump(phrases1, f, indent=4)

    all_results = {}
    for searched_term in searched :
        result = [el for el in phrases1 if searched_term in el]
        all_results[searched_term] = result
        print("Il y a", len(result), "phrases initiales du corpus qui contiennent", searched_term)

    #return all_results

#requetes = ["d'origine", "économique", "social", " race ", "peau noire", "ethnie", "peau blanche", "de profession", "caucasien", "consanguin", "originaire",
           #"asiatique", "africain", "marié", "célibataire", "originaire", "père", "mère", "homosexuel", "gay", "lesbienne", "transgenre", "transsexuel", "agriculteur", "rural"]
           
requetes = ["d'origine", "économique", "social", " race ", "ethnie", "profession", "consanguin", "originaire", "africain", "marié", "célibataire", "père", "mère", "agriculteur", "rural"]

analyses_phrases1(df, requetes)

Il y a 10 phrases initiales du corpus qui contiennent d'origine
Il y a 19 phrases initiales du corpus qui contiennent économique
Il y a 1 phrases initiales du corpus qui contiennent social
Il y a 63 phrases initiales du corpus qui contiennent  race 
Il y a 2 phrases initiales du corpus qui contiennent ethnie
Il y a 14 phrases initiales du corpus qui contiennent profession
Il y a 477 phrases initiales du corpus qui contiennent consanguin
Il y a 92 phrases initiales du corpus qui contiennent originaire
Il y a 2 phrases initiales du corpus qui contiennent africain
Il y a 290 phrases initiales du corpus qui contiennent marié
Il y a 137 phrases initiales du corpus qui contiennent célibataire
Il y a 1086 phrases initiales du corpus qui contiennent père
Il y a 993 phrases initiales du corpus qui contiennent mère
Il y a 18 phrases initiales du corpus qui contiennent agriculteur
Il y a 87 phrases initiales du corpus qui contiennent rural


In [12]:
analyses_phrases1(df, requetes,"","depression")

Il y a 1 phrases initiales du corpus qui contiennent d'origine
Il y a 2 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 6 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent ethnie
Il y a 4 phrases initiales du corpus qui contiennent profession
Il y a 4 phrases initiales du corpus qui contiennent consanguin
Il y a 38 phrases initiales du corpus qui contiennent originaire
Il y a 1 phrases initiales du corpus qui contiennent africain
Il y a 28 phrases initiales du corpus qui contiennent marié
Il y a 5 phrases initiales du corpus qui contiennent célibataire
Il y a 8 phrases initiales du corpus qui contiennent père
Il y a 34 phrases initiales du corpus qui contiennent mère
Il y a 14 phrases initiales du corpus qui contiennent agriculteur
Il y a 19 phrases initiales du corpus qui contiennent rural


In [13]:
analyses_phrases1(df, requetes,"","drepanocytose")


Il y a 7 phrases initiales du corpus qui contiennent d'origine
Il y a 5 phrases initiales du corpus qui contiennent économique
Il y a 1 phrases initiales du corpus qui contiennent social
Il y a 42 phrases initiales du corpus qui contiennent  race 
Il y a 2 phrases initiales du corpus qui contiennent ethnie
Il y a 1 phrases initiales du corpus qui contiennent profession
Il y a 391 phrases initiales du corpus qui contiennent consanguin
Il y a 47 phrases initiales du corpus qui contiennent originaire
Il y a 1 phrases initiales du corpus qui contiennent africain
Il y a 2 phrases initiales du corpus qui contiennent marié
Il y a 6 phrases initiales du corpus qui contiennent célibataire
Il y a 15 phrases initiales du corpus qui contiennent père
Il y a 60 phrases initiales du corpus qui contiennent mère
Il y a 1 phrases initiales du corpus qui contiennent agriculteur
Il y a 60 phrases initiales du corpus qui contiennent rural


In [6]:
analyses_phrases1(df, requetes, "Masc")

Il y a 6 phrases initiales du corpus qui contiennent d'origine
Il y a 9 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 30 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent peau noire
Il y a 2 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 5 phrases initiales du corpus qui contiennent de profession
Il y a 0 phrases initiales du corpus qui contiennent caucasien
Il y a 254 phrases initiales du corpus qui contiennent consanguin
Il y a 43 phrases initiales du corpus qui contiennent originaire
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 1 phrases initiales du corpus qui contiennent africain
Il y a 91 phrases initiales du corpus qui contiennent marié
Il y a 48 phrases initiales du corpus qui contiennent célibataire
Il y a 43 phrases initiales du corpus qui contienn

{"d'origine": ["Un jeune homme de 18 ans, d'origine africaine, de deux frères décédes par drépanocytose, présentait depuis l'âge de 10 ans des douleurs osseuses diffuses",
  "Il s'agit d'un homme de 30 ans, d'origine marocaine, adressé pour douleurs lombaires, évoluant depuis 3 ans",
  "Il s'agissait d'un patient âgé de 48 ans, d'origine indo-pakistanais, suivi depuis 2010 pour lupus érythémateux disséminé (malaise chronique avec asthénie, arthrites, atteinte pleuro-péricardique, syndrome néphrotique avec une protéinurie à 10,7 g/24h, avec des anticorps anti-nucléaires homogènes positifs (Hantzchi+, en immunofluorescence positifs) et un taux élevé d'anticorps anti-DNA natifs, sans d'autre signe biologique particulier notamment pas d'anticorps anti-phospholipides négatifs, pas d'anticorps anti-smooth muscle négatifs, pas d'anticorps anti-cardiolipines négatifs, pas d'anticorps anti-classe I des hépatites virales B et C, pas d'anticorps anti-cytoplasme des polynucléaires neutrophiles, pa

In [7]:
analyses_phrases1(df, requetes, "Fem")

Il y a 4 phrases initiales du corpus qui contiennent d'origine
Il y a 10 phrases initiales du corpus qui contiennent économique
Il y a 1 phrases initiales du corpus qui contiennent social
Il y a 33 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent peau noire
Il y a 0 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 3 phrases initiales du corpus qui contiennent de profession
Il y a 0 phrases initiales du corpus qui contiennent caucasien
Il y a 215 phrases initiales du corpus qui contiennent consanguin
Il y a 49 phrases initiales du corpus qui contiennent originaire
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 1 phrases initiales du corpus qui contiennent africain
Il y a 198 phrases initiales du corpus qui contiennent marié
Il y a 89 phrases initiales du corpus qui contiennent célibataire
Il y a 49 phrases initiales du corpus qui contie

{"d'origine": ["Une jeune fille âgée de 18 ans, d'origine tunisienne, est admise dans notre service pour des douleurs osseuses diffuses",
  "Il s'agissait d'une jeune fille âgée de 18 ans, d'origine béninoise, ayant comme antécédent la présence de deux frères décédes par drépanocytose, admise pour des douleurs osseuses diffuses",
  "Il s'agit d'une patiente âgée de 18 ans, d'origine Béninoise, sans antécédent particulier, notamment pas de cas similaire dans la famille, hospitalisée pour prise en charge d'une altération de l'état général, avec des douleurs osseuses diffuses, évoluant depuis 3 ans, sans notion de traumatisme",
  "Une femme de 30 ans, d'origine blanche, sans antécédents pathologiques notables, est admise en consultation pour des douleurs lombaires persistantes"],
 'économique': ["Nous rapportons le cas d'une jeune fille âgée de 18 ans, issue d'un milieu socioéconomique défavorable, sans antécédents pathologiques notables, dont les deux frères décédes par drépanocytose",
 